### Libraries

In [39]:
import torch
from torchvision import transforms
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import numpy as np
import requests
import os

In [45]:
images_folder_path = "../../1) Data_Collection/OID/Dataset/train"
output = "dinov2_embeddings"

In [53]:
# Load DINOv2 model (replace with the actual loading function or model URL)
def load_dinov2_model():
    model = AutoModel.from_pretrained("facebook/dinov2-base")
    model.eval()  # Set model to evaluation mode
    return model

# Preprocess the image
def preprocess_image(image):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    return transform(image).unsqueeze(0)  # Add batch dimension

# Generate embeddings
def generate_embeddings(model, image_tensor):
    with torch.no_grad():
        embeddings = model(image_tensor)
    return embeddings["last_hidden_state"].cpu().numpy()

# Save embeddings to a .bin file
def save_embeddings_to_bin(embeddings, file_path):
    embeddings.astype(np.float32).tofile(file_path)

In [58]:
# Main function
def main(image_path, output_path):
    model = load_dinov2_model()
    image_tensor = preprocess_image(image_path)
    embeddings = generate_embeddings(model, image_tensor)
    save_embeddings_to_bin(embeddings, output_path)

In [60]:
for one_category in os.listdir(images_folder_path):
    one_category_path = os.path.join(images_folder_path, one_category)

    all_images_in_category_folder = [x for x in os.listdir(one_category_path) if x.endswith("jpg")]

    for img in all_images_in_category_folder:

        img_path = os.path.join(one_category_path, img)

        image = Image.open(img_path)
        image = image.convert("RGB")
        
        output_path = os.path.join(output, img[:-3]+"bin")

        main(image, output_path)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
